In [1]:
from utilsJupyter import * 
from IPython.display import clear_output
import numpy as np
import math

Using TensorFlow backend.
/home/rose/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rose/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rose/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rose/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [8]:
"""
Setting hyperparameters and collecting data
"""
timeseries = 64
layers = int(np.log(timeseries)/np.log(2))-1
look_ahead = 1
batch_size = look_ahead

symbols = get_symbols()
symbols_to_int = {}
int_to_symbols = {}
dates = []
data = {}

for i in range(len(symbols)):
    symbol = symbols[i]
    symbols_to_int[symbol] = i
    int_to_symbols[i] = symbol


for symbol in symbols:
    print(symbol)
    df = pd.read_csv(f'./data/{symbol}.csv',index_col='datetime')
    
    df.columns = [c.capitalize() for c in df.columns]
    dates += [d.split(" ")[0] for d in list(df.index.values)]
    _df, next_data = prepare_data(df, look_ahead)
    #_df['symbol'] = [symbols_to_int[symbol]] * len(_df)
    data[symbol] = _df

dates = list(set(dates))
dates = [(date, int(date.replace('-',''))) for date in dates]
dates = sorted(dates, key=lambda x: x[1])
dates = [date[0] for date in dates]
clear_output()
print("Data Retrieved")


Data Retrieved


In [3]:
"""
Training against data
"""

nb_batches = math.floor((len(dates)-timeseries-look_ahead)/batch_size)

classifier = Classifier(timeseries, _df.shape[1]-3, layers, 1, len(data))

classifier.summary()

# _day_in_batch = np.array([i for i in range(batch_size)])
# pred_classes = {}
# day_in_batch = {}

# for symbol in symbols:
#     pred_classes[symbol] = []
#     day_in_batch[symbol] = []
    
# for j in range(nb_batches):
#     print(f'{j}/{nb_batches} batches')
#     start_date = dates[j*batch_size]
#     end_date = dates[(j+1)*batch_size+timeseries]
    
#     for symbol in data:
#         _df = data[symbol]
#         df_batch = _df.loc[start_date:end_date]
#         X,Y,Class = get_x_y(df_batch, timeseries)
#         X = X[:batch_size]
#         Y = Y[:batch_size]
#         Class = Class[:batch_size]
#         if(len(X)!=batch_size):
#             continue
#         symbol_index = symbols_to_int[symbol]
#         embedding_input = np.array([symbol_index] * batch_size)
#         pred_Class = classifier.predict([X, embedding_input, _day_in_batch])
        
#         _pred_Class = [_[0] for _ in pred_Class]
#         pred_classes[symbol] += _pred_Class
#         day_in_batch[symbol] += list(_day_in_batch)
        
#     for symbol in data:
#         _df = data[symbol]
#         df_batch = _df.loc[start_date:end_date]
#         X,Y,Class = get_x_y(df_batch, timeseries)
#         X = X[:batch_size]
#         Y = Y[:batch_size]
#         Class = Class[:batch_size]
#         if(len(X)!=batch_size):
#             continue
#         symbol_index = symbols_to_int[symbol]
#         embedding_input = np.array([symbol_index] * batch_size)
#         classifier.train_on_batch([X, embedding_input, _day_in_batch],Class)
        

#     if (j % 100 == 0) and j!=0:
#         clear_output()

# for symbol in symbols:
#     __df = data[symbol]
#     __df = __df.tail(len(pred_classes[symbol]))
#     __df['real_pred'] = pred_classes[symbol]
#     __df['day_in_batch'] = day_in_batch[symbol]
#     __df['was_right'] = np.array([int(_>.5) for _ in pred_classes[symbol]]) == __df['pred']
#     __df['was_right'] = [int(_) for _ in __df['was_right']]
#     data[symbol] = __df
# clear_output()
# print('Train and Test Complete')

tracking <tf.Variable 'trainable:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_1:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_2:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_3:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_4:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_5:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_6:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_7:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_8:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_9:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_10:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_11:0' shape=() dtype=float32> _trainable_tensor
tracking <tf.Variable 'trainable_12:0' s

In [4]:
for symbol in data:
    __df = data[symbol]
    __df.to_csv(f'./results/{symbol}.csv')
print("Results Saved")

Results Saved


In [5]:
(__df['pred'] == 1).tail(1000).mean()

0.4521158129175947

In [7]:
__df['pred'] == 1

datetime
2020-08-13 15:20:00     True
2020-08-13 15:40:00     True
2020-08-13 16:00:00    False
2020-08-13 16:20:00    False
2020-08-13 16:40:00     True
                       ...  
2020-08-21 19:00:00     True
2020-08-21 19:20:00    False
2020-08-21 19:40:00     True
2020-08-21 20:00:00    False
2020-08-21 20:20:00    False
Name: pred, Length: 449, dtype: bool